In [ ]:
import pandas as pd
import numpy as np

In [ ]:
alkamel_timing_data = pd.read_csv('timing_data.csv', index_col=0)
alkamel_teams = pd.read_csv('teams.csv', index_col = 0)

In [ ]:
alkamel_circuits = alkamel_teams.groupby('circuit').mean().reset_index()
alkamel_teams['event'] = alkamel_teams['circuit']
alkamel_circuits['event'] = alkamel_circuits['circuit'].str.split(" ")

#okay, some of these in 'circuit' should be event name, and we should fix the circuit to make sure we only grab the correct circuit
def capitalize_circuit(x):
    output = ""
    for y in x:
        output = output + " " + str.capitalize(y)
    return str.strip(output)
alkamel_circuits['event'] = alkamel_circuits['event'].apply(capitalize_circuit)
alkamel_circuits.to_csv('circuits.csv')

#manually fixed the circuits here

In [ ]:
#fixing the circuit for team
def fix_circuit():
    fixed_circuits = pd.read_csv("circuits.csv", index_col=0)
    alkamel_teams = pd.read_csv('teams.csv', index_col = 0)
    fixed_circuits = fixed_circuits.drop(['number', 'round'], axis=1)
    alkamel_teams['event'] = alkamel_teams['circuit']
    alkamel_teams = alkamel_teams.drop(['circuit'], axis=1)
    pd.merge(alkamel_teams, fixed_circuits, on = 'event').to_csv('teams.csv')

In [ ]:
alkamel_timing_data = pd.read_csv('timing_data.csv', index_col=0)
alkamel_teams = pd.read_csv('teams.csv', index_col = 0)
alkamel_data = pd.merge(alkamel_timing_data, alkamel_teams, on = 'key')

In [ ]:
#sort data by key + elapsed
alkamel_data.sort_values(['key', 'elapsed_seconds']).reset_index(drop=True).head(50000).to_csv('test.csv')

In [ ]:
#I already have fia wec data in this, so maybe I might append it to the bottom of it after I get the key?...
#oh no nvm I have to keep the data bc it's only race data ugh
#I guess I can just figure out how to do the other sessions, let's focus on that first

#let's figure out how to do this with only one championship first
fia_wec_rest_of_data = alkamel_data[(alkamel_data['championship'] == "FIA WEC")].reset_index(drop=True)
fia_wec_rest_of_data

In [ ]:
fia_wec_rest_of_data = fia_wec_rest_of_data.sort_values(['key', 'elapsed_seconds']).reset_index(drop=True)
fia_wec_rest_of_data

In [ ]:
#for the rest of the sessions... position is determined by the laps
#gotta determine position by lap number?... how do we determine this
fia_wec_r1 = fia_wec_rest_of_data[(fia_wec_rest_of_data['round'] == 1) & (fia_wec_rest_of_data['season'] == "2012") \
    & (fia_wec_rest_of_data['session'] != 'RACE')]
fia_wec_r1

In [ ]:
#okay, so let's spend some time trying to figure out how to loop thru data
#lap key methods
def search_lap_key_in_dict(key, dict):
    for d in dict:
        if d['lap_key'] == key:
            return d
    return None
def update_lap_value_in_dict(key, dict, col, value):
    dict_key = search_lap_key_in_dict(key, dict)
    dict_key.update({col : value})

#car key methods
def search_car_key_in_dict(key, dict):
    for d in dict:
        if d['car_key'] == key:
            return d
    return None
def update_car_value_in_dict(key, dict, col, value):
    dict_key = search_car_key_in_dict(key, dict)
    dict_key.update({col : value})

#create array based off of column
def create_array_from_column(column, df):
    wanted_columns = [column]
    array_groupby = df.groupby(column).mean()
    array_groupby = array_groupby.reset_index()
    array = array_groupby[wanted_columns]
    array = np.array(array)
    return array

def update_timing_board_faster(dictionary):
    dict_df = pd.DataFrame.from_dict(dictionary)
    return dict_df

#slow updating of timing
def update_timing_board(dictionary):
    dict_df = pd.DataFrame.from_dict(dictionary)
    timing_board = dict_df.sort_values('fastest_lap_seconds')
    timing_board.position = range(1, len(timing_board)+1)
    #sort to get class position
    timing_board.sort_values(['class', 'fastest_lap_seconds'])
    #separate by class
    timing_board_classes = create_array_from_column('class', timing_board)
    class_counts = timing_board.groupby('class')['car_key'].count()
    timing_board_with_class_position = pd.DataFrame(columns = timing_board.columns)
    for c in timing_board_classes:
        class_timing = timing_board[timing_board['class'] == c[0]]
        class_timing.class_position = range(1, class_counts.at[c[0]]+1)
        timing_board_with_class_position = pd.concat([timing_board_with_class_position, class_timing])
    return timing_board_with_class_position

championships = create_array_from_column('championship', alkamel_data)
#filter data through championship
for c in championships:
    if c == 'FIA WEC':
        championship_data = alkamel_data[alkamel_data['championship'] == c[0]]
        #filter data through each season
        seasons = create_array_from_column('season', championship_data)
        for s in seasons:
            if s == '2012':
                season_data = championship_data[championship_data['season'] == s[0]]
                #filter data through each round
                rounds = create_array_from_column('round', season_data)
                for r in rounds:
                    if r == 1:
                        round_data = season_data[season_data['round'] == r[0]]
                        #filter data through each session
                        sessions = create_array_from_column('session', round_data)
                        for s2 in sessions:
                            session_data = round_data[round_data['session'] == s2[0]]
                            #sort by elapsed
                            session_data = session_data.sort_values('elapsed_seconds').reset_index(drop=True)
                            #create lap key
                            session_data['car_key'] = session_data['season'] + "_" + session_data['round'].map(str) + "_" + \
                            session_data['session'] + "_" + session_data['number'].map(str)
                            session_data['position'] = 0
                            session_data['class_position'] = 0
                            wanted_columns = ['car_key', 'class']
                            lap_keys = session_data.groupby(['car_key', 'class']).mean().sort_values(by='elapsed_seconds').reset_index()
                            lap_keys = lap_keys[wanted_columns]
                            lap_keys['position'] = 0
                            lap_keys['class_position'] = 0
                            lap_keys['fastest_lap_seconds'] = 0
                            lap_keys['last_lap_seconds'] = 0
                            lap_keys_dict = lap_keys.to_dict('records')
                            for i in range(0, len(session_data)):
                                x = session_data.iloc[i]
                                #updates lap time
                                if(x['crossing_finish_line_in_pit'] != 'B'):
                                    current_car_timing = search_car_key_in_dict(x['car_key'], lap_keys_dict)
                                    if(current_car_timing['fastest_lap_seconds'] > x['lap_time_seconds'] or current_car_timing['fastest_lap_seconds'] == 0):
                                        update_car_value_in_dict(x['car_key'], lap_keys_dict, 'fastest_lap_seconds', x['lap_time_seconds'])
                                    update_car_value_in_dict(x['car_key'], lap_keys_dict, 'last_lap_seconds', x['lap_time_seconds'])
                                if x['lap_time_seconds'] == 0:
                                    x.at[i, 'position'] = 0
                                    x.at[i, 'class_position'] = 0
                                else:
                                    lap_keys_dict = update_timing_board(lap_keys_dict).to_dict('records')
                                    session_data.loc[i, 'position'] = search_car_key_in_dict(x['car_key'], lap_keys_dict)['position']
                                    session_data.loc[i, 'class_position'] = search_car_key_in_dict(x['car_key'], lap_keys_dict)['class_position']
session_data

In [ ]:


#let's use this as proof of concept
session_data = alkamel_data[(alkamel_data['round'] == 1)&(alkamel_data['championship'] == 'FIA WEC')\
    &(alkamel_data['season']=='2012')&(alkamel_data['session']=='FREE PRACTICE 1')].reset_index(drop=True)

#sort the data by elapsed_seconds
session_data = session_data.sort_values('elapsed_seconds').reset_index(drop=True)

#push all the records in the session to a dict
session_data['lap_key'] = session_data['season'] + "_" + session_data['round'].map(str) + "_" + \
session_data['session'] + "_" + session_data['number'].map(str) + "_" + session_data['lap_number'].map(str)
session_data['car_key'] = session_data['season'] + "_" + session_data['round'].map(str) + "_" + \
session_data['session'] + "_" + session_data['number'].map(str)
session_data['position'] = 0
session_data['class_position'] = 0
wanted_columns = ['car_key', 'class']
lap_keys = session_data.groupby(['car_key', 'class']).mean().sort_values(by='elapsed_seconds').reset_index()
lap_keys = lap_keys[wanted_columns]
lap_keys['position'] = 0
lap_keys['class_position'] = 0
lap_keys['fastest_lap_seconds'] = 0
lap_keys['last_lap_seconds'] = 0
lap_keys_dict = lap_keys.to_dict('records')

for i in range(0, len(session_data)):
    x = session_data.iloc[i]
    #updates lap time
    if(x['crossing_finish_line_in_pit'] != 'B'):
        current_car_timing = search_car_key_in_dict(x['car_key'], lap_keys_dict)
        if(current_car_timing['fastest_lap_seconds'] > x['lap_time_seconds'] or current_car_timing['fastest_lap_seconds'] == 0):
            update_car_value_in_dict(x['car_key'], lap_keys_dict, 'fastest_lap_seconds', x['lap_time_seconds'])
        update_car_value_in_dict(x['car_key'], lap_keys_dict, 'last_lap_seconds', x['lap_time_seconds'])
    if x['lap_time_seconds'] == 0:
        x.at[i, 'position'] = 0
        x.at[i, 'class_position'] = 0
    else:
        lap_keys_dict = update_timing_board(lap_keys_dict).to_dict('records')
        session_data.loc[i, 'position'] = search_car_key_in_dict(x['car_key'], lap_keys_dict)['position']
        session_data.loc[i, 'class_position'] = search_car_key_in_dict(x['car_key'], lap_keys_dict)['class_position']
dict_df = pd.DataFrame.from_dict(lap_keys_dict)
session_data

In [ ]:

#loop through the timing to get the fastest lap
for i in range(0, len(session_data)):
    x = session_data.iloc[i]
    #updates lap time
    if(x['crossing_finish_line_in_pit'] != 'B'):
        current_car_timing = search_key_in_dicts(x, x['car_key'], lap_keys_dict)
        if(current_car_timing['fastest_lap_seconds'] > x['lap_time_seconds'] or current_car_timing['fastest_lap_seconds'] == 0):
            update_value_in_dicts(x['car_key'], lap_keys_dict, 'fastest_lap_seconds', x['lap_time_seconds'])
    #update position of the current lap
    #if lap time is 0, put position as 0
#cumsum?...
#sort to get position
session_data